<a href="https://colab.research.google.com/github/sgardziewicz/JsonToXmlGeneratorForCoinmarketcap/blob/master/Colab_Environment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#do colaba  i wtedy tweet.username
!pip install snscrape
!pip install sentencepiece
!pip install emoji
!pip install transformers

In [1]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import torch
import transformers
from transformers import pipeline
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification

# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

pd.options.display.max_rows = 4000

Using device: cuda

GeForce GTX 1060 6GB
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [14]:
#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

GeForce GTX 1060 6GB
Memory Usage:
Allocated: 0.5 GB
Cached:    3.3 GB


In [34]:
%%time
tweets_list = []

for i, tweet in enumerate(sntwitter.TwitterHashtagScraper('cyberpunk2077').get_items()):
  if i>10000:
    break
  tweets_list.append([tweet.date, tweet.id, tweet.content, tweet.username])

tweets_df = pd.DataFrame(tweets_list, columns=['Datetime', 'Tweet Id', 'Text', 'Username'])

CPU times: user 3.33 s, sys: 116 ms, total: 3.45 s
Wall time: 1min 28s


In [5]:
tweetsList = tweets_df['Text'].tolist()

In [6]:
tweetsList[0:5]

['Il est enfin disponible ! \nLe Nouveau #Trailer de CYBER ZOO sur @PlanetZooGame !\n\nThis is Now Available ON WORKSHOP ! \n\nCYBER ZOO (Official Trailer) -https://t.co/pObN476qdn\n\n#Youtube #Planetzoo #CYberpunk2077 #CyberZoo',
 'Life Goes On by Ryan Maggs via /r/Cyberpunk by u/ N7CombatWombat #cyberpunk #cyberpunk2077 #gaming https://t.co/K8nmnn6gjw',
 'Apparently I have 78 manual saves already... Idk why I keep doing it. #Cyberpunk2077',
 '#PS5Share, #Cyberpunk2077 https://t.co/n62FGa1Zgw',
 '@ick_hoer_dir___ @StockWatchPL Wie es aktuell ist, weiß ich nicht. Besitze die #PS4 Version nicht mehr. Zum Release, war es aber Katastrophal. Spiele #Cyberpunk2077 ausschließlich über Geforce NOW.']

In [7]:
listWithoutNewLine=[]
for item in tweetsList:
  listWithoutNewLine.append(item.replace('\n', ' '))

In [8]:
len(listWithoutNewLine)

10001

In [ ]:
def extractLists(lst): 
    return [[element] for element in lst]              

In [ ]:
import re
d=[]
pattern = re.compile('[^0-9A-Za-z ,]')
for item in tweetsList:
  d.append(pattern.sub('', item))

print(d[0:5])

['Check out SteelSeries Arctis 1 Wireless Gaming Headset Cyberpunk Edition  Used w Box  httpstcoU5ImIwNGyt via eBay eBay Xbox SteelSeries SteelSeries Cyberpunk2077 resale liquidation Bellingham supportsmallbusiness headsets', 'PS5Share, Cyberpunk2077 httpstcoZPCsuBAlbk', 'PS5Share, Cyberpunk2077 httpstco0mYuTxbwZr', 'CYBERPUNK 2077 EVERY BREATH YOU TAKE PART 42 is now uploaded httpstconCeGpTsLdd Cyberpunk2077 CDProjektRed PS5 Xbox Cyberpunk httpstconubBQ2LGTD', 'Spoiler free No es por nada pero de 3 finales en cyberpunk que ya he pasadode momento el ms creble para m  y con el que ms me siento identificado es el final malo Debera preocuparmecyberpunk2077 cyberpunk httpstcoZbX1T4NE8E']


In [9]:
import re        ## usuniecie ruskich itp
listWithoutNewLineWithoutUserNames = []
for item in listWithoutNewLine:
  listWithoutNewLineWithoutUserNames.append(re.sub('@[\w]+','',item).encode('ascii', errors='ignore').decode())

print(listWithoutNewLineWithoutUserNames[0:5])

['Il est enfin disponible !  Le Nouveau #Trailer de CYBER ZOO sur  !  This is Now Available ON WORKSHOP !   CYBER ZOO (Official Trailer) -https://t.co/pObN476qdn  #Youtube #Planetzoo #CYberpunk2077 #CyberZoo', 'Life Goes On by Ryan Maggs via /r/Cyberpunk by u/ N7CombatWombat #cyberpunk #cyberpunk2077 #gaming https://t.co/K8nmnn6gjw', 'Apparently I have 78 manual saves already... Idk why I keep doing it. #Cyberpunk2077', '#PS5Share, #Cyberpunk2077 https://t.co/n62FGa1Zgw', '  Wie es aktuell ist, wei ich nicht. Besitze die #PS4 Version nicht mehr. Zum Release, war es aber Katastrophal. Spiele #Cyberpunk2077 ausschlielich ber Geforce NOW.']


In [10]:
import re
listWithoutNewLineWithoutUserNamesWithoutHashtags = []
for item in listWithoutNewLineWithoutUserNames:
  listWithoutNewLineWithoutUserNamesWithoutHashtags.append(re.sub('#[\w]+','',item))

print(listWithoutNewLineWithoutUserNamesWithoutHashtags[0:5])

['Il est enfin disponible !  Le Nouveau  de CYBER ZOO sur  !  This is Now Available ON WORKSHOP !   CYBER ZOO (Official Trailer) -https://t.co/pObN476qdn     ', 'Life Goes On by Ryan Maggs via /r/Cyberpunk by u/ N7CombatWombat    https://t.co/K8nmnn6gjw', 'Apparently I have 78 manual saves already... Idk why I keep doing it. ', ',  https://t.co/n62FGa1Zgw', '  Wie es aktuell ist, wei ich nicht. Besitze die  Version nicht mehr. Zum Release, war es aber Katastrophal. Spiele  ausschlielich ber Geforce NOW.']


In [ ]:
tokenizerBertweet = AutoTokenizer.from_pretrained("cardiffnlp/bertweet-base-sentiment")          ##############################

#tokenizerBertweet = AutoTokenizer.from_pretrained("vinai/bertweet-base", normalization=True, max_length=128)
#modelBertweet = AutoModel.from_pretrained("vinai/bertweet-base")

modelBertweet = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/bertweet-base-sentiment")

analyzerBertweet = pipeline('sentiment-analysis', model=modelBertweet, tokenizer=tokenizerBertweet, device=0)
modelBertweet.cuda()

In [14]:
print( len(max(listWithoutNewLine, key=len)))

458


In [15]:
listWithoutNewLine[0:5]
print( len(max(listWithoutNewLineWithoutUserNamesWithoutHashtags, key=len)))

309


In [16]:
print( max(listWithoutNewLineWithoutUserNamesWithoutHashtags, key=len))

More changes I noticed: * They have toned down the ads/ put them on mute in most parts of the city. It's a good change IMO some ads r annoying. * Stash finally works as intended * Even quicker load times * It rains in NC now, tho weather isn't fully fixed yet.  https://t.co/MChjUStSEt https://t.co/dom2MiqW3w


In [ ]:
listWithoutNewLineWithoutUserNamesWithoutHashtags[44]

'    ,        .            ,      -         .'

In [ ]:
rusek = listWithoutNewLineWithoutUserNamesWithoutHashtags[44]

In [ ]:
print(rusek.encode("ascii", "ignore"))

b'    ,        .            ,      -         .'


In [ ]:
xd = tokenizerBertweet(listWithoutNewLineWithoutUserNamesWithoutHashtags[44])

In [ ]:
xd.tokens

<bound method BatchEncoding.tokens of {'input_ids': [0, 50354, 28570, 11973, 24229, 18511, 3, 62359, 60600, 13526, 59407, 13526, 24229, 17234, 61758, 61581, 59407, 13526, 60399, 7, 58065, 21276, 11973, 21610, 20890, 38099, 17234, 13526, 63103, 13526, 62359, 17234, 33464, 62359, 20890, 11973, 54283, 29013, 11973, 29013, 13384, 17234, 54283, 17234, 28321, 27901, 21276, 34828, 63103, 11973, 27901, 4, 61685, 13384, 58065, 13384, 21276, 61198, 60912, 55256, 27901, 13526, 63103, 29716, 61758, 61581, 21276, 13526, 53840, 58065, 21276, 18511, 21610, 13384, 29716, 13526, 13384, 54283, 24229, 38099, 24229, 21276, 13384, 29013, 43827, 29716, 18511, 13526, 28570, 11973, 60399, 28321, 56208, 58065, 13384, 21276, 21610, 11973, 17234, 13526, 63103, 13384, 29013, 7, 28321, 24229, 21610, 20890, 13526, 20890, 61198, 43827, 17234, 33464, 29013, 11973, 60399, 60695, 34, 54291, 20890, 11973, 29013, 60695, 56332, 20890, 38099, 17234, 28321, 3, 20890, 60695, 63103, 33464, 27901, 28570, 13526, 24229, 18511, 6

In [ ]:
tokenxs = xd.data['input_ids']

In [ ]:
tokenizerBertweet.batch_decode(tokenxs)

In [12]:
#import gc

#gc.collect()

torch.cuda.empty_cache()

In [28]:
finalList = listWithoutNewLineWithoutUserNamesWithoutHashtags

In [30]:
listdf = pd.DataFrame(finalList)

In [34]:
listdf.to_csv('list.csv', index=False)

In [3]:
odzysk = pd.read_csv('listTest10k.csv')

In [4]:
listaOdzysk = odzysk.values.tolist()
print(len(listaOdzysk))

9964


In [5]:
listaOdzysk[0:5]

[['Il est enfin disponible !  Le Nouveau  de CYBER ZOO sur  !  This is Now Available ON WORKSHOP !   CYBER ZOO (Official Trailer) -https://t.co/pObN476qdn     '],
 ['Life Goes On by Ryan Maggs via /r/Cyberpunk by u/ N7CombatWombat    https://t.co/K8nmnn6gjw'],
 ['Apparently I have 78 manual saves already... Idk why I keep doing it. '],
 [',  https://t.co/n62FGa1Zgw'],
 ['  Wie es aktuell ist, wei ich nicht. Besitze die  Version nicht mehr. Zum Release, war es aber Katastrophal. Spiele  ausschlielich ber Geforce NOW.']]

In [6]:
flat_list = [item for sublist in listaOdzysk for item in sublist]


In [7]:
len(flat_list)

9964

In [8]:
cleanedList = [x for x in flat_list if str(x) != 'nan']
print(len(cleanedList))

9962


In [15]:
%%time
scores=[]
scores.append(analyzerBertweet(cleanedList[0:500]))
scores.append(analyzerBertweet(cleanedList[501:1000]))
scores.append(analyzerBertweet(cleanedList[1001:1500]))
scores.append(analyzerBertweet(cleanedList[1501:2000]))
scores.append(analyzerBertweet(cleanedList[2001:2500]))
scores.append(analyzerBertweet(cleanedList[2501:3000]))
scores.append(analyzerBertweet(cleanedList[3001:3500]))
scores.append(analyzerBertweet(cleanedList[3501:4000]))
scores.append(analyzerBertweet(cleanedList[4001:4500]))
scores.append(analyzerBertweet(cleanedList[4501:5000]))


Wall time: 33.6 s


In [16]:
print(len(scores))
print(scores)

10
[[{'label': 'LABEL_1', 'score': 0.5768522024154663}, {'label': 'LABEL_1', 'score': 0.9233905673027039}, {'label': 'LABEL_0', 'score': 0.529954731464386}, {'label': 'LABEL_1', 'score': 0.9184607267379761}, {'label': 'LABEL_1', 'score': 0.9005232453346252}, {'label': 'LABEL_1', 'score': 0.9335196614265442}, {'label': 'LABEL_1', 'score': 0.8958781957626343}, {'label': 'LABEL_2', 'score': 0.7346646189689636}, {'label': 'LABEL_1', 'score': 0.9277018308639526}, {'label': 'LABEL_1', 'score': 0.9094046354293823}, {'label': 'LABEL_1', 'score': 0.6918765902519226}, {'label': 'LABEL_1', 'score': 0.5548128485679626}, {'label': 'LABEL_1', 'score': 0.9033317565917969}, {'label': 'LABEL_1', 'score': 0.939208447933197}, {'label': 'LABEL_1', 'score': 0.9267732501029968}, {'label': 'LABEL_1', 'score': 0.9106860160827637}, {'label': 'LABEL_1', 'score': 0.9200596213340759}, {'label': 'LABEL_1', 'score': 0.9138519763946533}, {'label': 'LABEL_0', 'score': 0.4663352072238922}, {'label': 'LABEL_0', 'score'

In [17]:
flat_scores = [item for sublist in scores for item in sublist]
print(len(flat_scores))

4991


In [26]:
flat_list[5047:5050]

['Details from my male V portrait   https://t.co/8LJRAQd8gV',
 nan,
 'Some art of my Male V, Theo, from   Reference was a photo of who his in-game appearance is actually based on, model Nils Kuiper. My V, with and without cybernetics, as well as his natural hair colour  art done for fun, not for sale anywhere.   https://t.co/CRka5rz7qX']

In [ ]:
# Z VIN
from transformers import AutoModel
device = "cuda:0"
bertweet = AutoModel.from_pretrained("vinai/bertweet-base")
bertweet = bertweet.to(device)

tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", normalization=True, use_fast=False)

line = "SC has first two presumptive cases of coronavirus, DHEC confirms https://postandcourier.com/health/covid19/sc-has-first-two-presumptive-cases-of-coronavirus-dhec-confirms/article_bddfe4ae-5fd3-11ea-9ce4-5f495366cee6.html?utm_medium=social&utm_source=twitter&utm_campaign=user-share… via @postandcourier"

input_ids = torch.tensor([tokenizer.encode(line)])
input_ids = input_ids.to(device)
with torch.no_grad():
    features = bertweet(input_ids)


#tokenizerBertweetVin = AutoTokenizer.from_pretrained("cardiffnlp/bertweet-base-sentiment")
#modelBertweetVin = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/bertweet-base-sentiment")
#analyzerBertweetVin = pipeline('sentiment-analysis', model=modelBertweet, tokenizer=tokenizerBertweet, device=0)
#modelBertweetVin.cuda()

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [ ]:
input_ids

tensor([[    0,  4040,    90,   160,   255, 35006, 26940,  2612,    15,  1456,
             7,   429,  6814,   499, 12952,    10,    28,   156,     5,     2]],
       device='cuda:0')

In [ ]:
features

BaseModelOutputWithPoolingAndCrossAttentions([('last_hidden_state',
                                               tensor([[[-0.0105,  0.2210,  0.1384,  ...,  0.0587,  0.0552, -0.1328],
                                                        [ 0.1169,  0.0007,  0.2255,  ...,  0.0935, -0.2912,  0.4768],
                                                        [-0.1160,  0.2040, -0.2274,  ...,  0.0782, -0.1798, -0.2573],
                                                        ...,
                                                        [-0.0444, -0.2010, -0.1522,  ..., -0.1283, -0.0186, -0.3401],
                                                        [ 0.1676, -0.2911, -0.0221,  ..., -0.1167,  0.1905, -0.0077],
                                                        [-0.0266,  0.2417,  0.1463,  ...,  0.0332,  0.0965, -0.1207]]],
                                                      device='cuda:0')),
                                              ('pooler_output',
                        

In [ ]:
inputy = tokenizer("good job", return_tensors="pt")
outputy = bertweet(**inputy)

RuntimeError: ignored

In [ ]:
hidden_states = outputy[1]
pooled_output = torch.cat(tuple([hidden_states[i] for i in [-4, -3, -2, -1]]), dim=-1)
pooled_output = pooled_output[:, 0, :]
pooled_output = self.dropout(pooled_output)
# classifier of course has to be 4 * hidden_dim, because we concat 4 layers
logits = self.classifier(pooled_output)

IndexError: ignored

In [ ]:
features.last_hidden_state.shape

torch.Size([1, 20, 768])

In [ ]:
%%time
analyzerBertweetVin(d)

In [ ]:
%%time
analyzerBertweet(d)

In [ ]:
%%time
scores = []
i=0
for item in extractLists(d):
  i=i+1
  if i>50:
    break
  scores.append(analyzerBertweet(item))

CPU times: user 813 ms, sys: 58 ms, total: 871 ms
Wall time: 875 ms


In [ ]:
#torch.cuda.empty_cache()

In [ ]:
%%time
scoresTests = [] 
scoresTests = analyzerBertweet(d)                   # git

CPU times: user 3.13 s, sys: 2.46 s, total: 5.59 s
Wall time: 5.57 s


In [18]:
scores_df = pd.DataFrame(flat_scores)         #git

In [19]:
scores_df.head()

,label,score
0,LABEL_1,0.576852
1,LABEL_1,0.923391
2,LABEL_0,0.529955
3,LABEL_1,0.918461
4,LABEL_1,0.900523


In [20]:
scores_df[80:95]

,label,score
80,LABEL_1,0.905955
81,LABEL_1,0.852510
82,LABEL_1,0.911074
83,LABEL_2,0.679532
84,LABEL_1,0.910787
85,LABEL_1,0.934858
86,LABEL_0,0.797494
87,LABEL_1,0.898279
88,LABEL_2,0.956453
89,LABEL_1,0.801019


In [ ]:
d[80:95]

['lokonda1904 DasDoHD TorstenWieland Kein halbwegs seriser Kandidat konnte nach der Rangnick Nummer im',
 'Schalke htte heute den ultimativen Aprilscherz bringen knnen, indem sie melden, dass unserr allseits',
 'peter knbel war brigens noch am 1003 abend im schweizer fernsehen als champions leaguekommentator zu',
 'Beim S04 Spiel eindeutig die Karte, die hier fr die grsste Erheiterung sorgte httpstcobLp9tJgNbD',
 'S04  httpstcoYedXkpK6PT',
 'Buchta muss immer noch weg  S04',
 '      Transfermarkt     15   11     75  55     7  5     7  4   Oczipka 15   800  S04 httpstcofE1ONkM',
 'PlanetaryIntel ob die Millionenhilfe fr S04 nicht vielleicht in den Sand gesetzt war',
 'Oh S04 httpstcoHdQFrzdmai',
 'Samstag 1615 Sky einschalten Ich bin in der Liveschalte zu sehenhren S04',
 'Heute gehe ich inhaltlich mal fremd Um 1904 Uhr stellen wir uns die Frage Ist S04 noch zu retten Die',
 'Hoffentlich schafft Schalke noch das Wunder und bleibt drin s04',
 'microbrick Geil Wrde S04 gut stehen ',
 'S04

In [21]:
scores_df['label'].value_counts()

LABEL_1    3924
LABEL_2     773
LABEL_0     294
Name: label, dtype: int64

In [ ]:
# Creating list to append tweet data to
tweets_list2 = []

# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('its the elephant since:2020-06-01 until:2020-07-31').get_items()):
    if i>10:
        break
    tweets_list2.append([tweet.date, tweet.id, tweet.content, tweet.username])

    
# Creating a dataframe from the tweets list above
tweets_df2 = pd.DataFrame(tweets_list2, columns=['Datetime', 'Tweet Id', 'Text', 'Username'])

In [ ]:
%%time
device = "cuda:0"
modelBertweet = modelBertweet.to(device)

line = "its absolutely great"
input_ids = torch.tensor([tokenizerBertweet.encode(line)])
input_ids = input_ids.to(device)
with torch.no_grad():
    features = modelBertweet(input_ids) # Models outputs are now tuples


CPU times: user 16.6 ms, sys: 0 ns, total: 16.6 ms
Wall time: 18.8 ms


In [ ]:
features

BaseModelOutputWithPoolingAndCrossAttentions([('last_hidden_state',
                                               tensor([[[-0.0711,  0.2637,  0.2211,  ..., -0.0843, -0.0354, -0.0862],
                                                        [-0.0318, -0.1466, -0.1101,  ..., -0.3499,  0.0830,  0.0827],
                                                        [-0.1999, -0.3070,  0.0993,  ..., -0.2399,  0.0141,  0.1390],
                                                        [-0.1128,  0.2849,  0.2530,  ..., -0.1139, -0.0324, -0.0858]]],
                                                      device='cuda:0')),
                                              ('pooler_output',
                                               tensor([[ 2.6828e-01, -1.6531e-01,  1.0120e-01, -2.1013e-01,  7.9331e-02,
                                                        -1.4480e-01,  8.6310e-02, -1.3911e-01,  1.8340e-01, -8.2742e-02,
                                                        -1.0040e-01,  4.6939e-0

In [ ]:
torch.argmax(features['logits'])